# Knet CNN Example

In [11]:
using Pkg; haskey(Pkg.installed(),"Knet") || Pkg.add("Knet")
using Knet
True=true # so we can read the python params
include("common/params.py")
gpu()

0

In [25]:
println("OS: ", Sys.KERNEL)
println("Julia: ", VERSION)
println("Knet: ", Pkg.installed()["Knet"])
println("GPU: ", replace(read(`nvidia-smi --query-gpu=name --format=csv,noheader`,String),'\n'=>", "))

OS: Linux
Julia: 1.0.0
Knet: 1.0.1+
GPU: TITAN Xp, TITAN Xp, TITAN Xp, TITAN Xp, TITAN Xp, TITAN Xp, TITAN Xp, TITAN Xp, 


In [13]:
# define model
function initmodel(; atype=KnetArray, dtype=Float32, winit=xavier, binit=zeros)
    w(dims...)=atype(winit(dtype,dims...))
    b(dims...)=atype(binit(dtype,dims...))
    return Any[
        w(3,3,3,50), b(1,1,50,1),
        w(3,3,50,50), b(1,1,50,1),
        w(3,3,50,100), b(1,1,100,1),
        w(3,3,100,100), b(1,1,100,1),
        w(512,6400), b(512,1),
        w(10,512), b(10,1)
    ]
end;

In [31]:
# define loss and its gradient
function predict(w,x; pdrop=(0,0))
    convbias(x,w,b) = conv4(w,x;padding=1) .+ b
    fc(x,w,b) = w * mat(x) .+ b;
    x = relu.(convbias(x,w[1],w[2]))
    x = relu.(pool(convbias(x,w[3],w[4])))
    x = dropout(x,pdrop[1])
    x = relu.(convbias(x,w[5],w[6]))
    x = relu.(pool(convbias(x,w[7],w[8])))
    x = dropout(x,pdrop[1])
    x = relu.(fc(x,w[9],w[10]))
    x = dropout(x,pdrop[2])
    return fc(x,w[11],w[12])
end

loss(w,x,y;o...)=nll(predict(w,x;o...),y) # nll: negative log likelihood
lossgradient = grad(loss);

In [32]:
# load data
include(Knet.dir("data","cifar.jl"))
@time (xtrn,ytrn,xtst,ytst,lbls)=cifar10()
dtrn = minibatch(xtrn,ytrn,BATCHSIZE,shuffle=true,xtype=KnetArray)
dtst = minibatch(xtst,ytst,BATCHSIZE,shuffle=false,xtype=KnetArray)
for d in (xtrn,ytrn,xtst,ytst); println(summary(d)); end

  0.070144 seconds (67.62 k allocations: 3.424 MiB)
32×32×3×50000 Array{Float32,4}
50000-element Array{UInt8,1}
32×32×3×10000 Array{Float32,4}
10000-element Array{UInt8,1}


In [33]:
# prepare for training
model = optim = nothing; Knet.gc() # Clear memory from last run
model = initmodel()
optim = optimizers(model, Momentum; lr=LR, gamma=MOMENTUM);

In [34]:
# cold start
@time for (x,y) in dtrn
    grads = lossgradient(model, x, y; pdrop=(0.25,0.5))
    update!(model, grads, optim)
end

  6.474056 seconds (2.00 M allocations: 670.819 MiB, 1.59% gc time)


In [35]:
# prepare for training
model = optim = nothing; Knet.gc() # Clear memory from last run
model = initmodel()
optim = optimizers(model, Momentum; lr=LR, gamma=MOMENTUM);

In [36]:
# 159s
@info("Training...")
@time for epoch in 1:EPOCHS
    @time for (x,y) in dtrn
        grads = lossgradient(model, x, y; pdrop=(0.25,0.5))
        update!(model, grads, optim)
    end
end

┌ Info: Training...
└ @ Main In[36]:2


  5.831162 seconds (1.88 M allocations: 664.773 MiB, 1.54% gc time)
  4.364551 seconds (1.88 M allocations: 664.628 MiB, 2.05% gc time)
  4.277376 seconds (1.88 M allocations: 664.628 MiB, 2.06% gc time)
  4.452553 seconds (1.88 M allocations: 664.628 MiB, 1.99% gc time)
  4.311131 seconds (1.88 M allocations: 664.628 MiB, 1.93% gc time)
  4.310649 seconds (1.88 M allocations: 664.628 MiB, 2.09% gc time)
  4.327310 seconds (1.88 M allocations: 664.628 MiB, 1.92% gc time)
  4.363336 seconds (1.88 M allocations: 664.628 MiB, 2.22% gc time)
  4.308050 seconds (1.88 M allocations: 664.626 MiB, 1.98% gc time)
  4.379973 seconds (1.88 M allocations: 664.626 MiB, 1.94% gc time)
 44.936848 seconds (18.84 M allocations: 6.491 GiB, 1.96% gc time)


In [37]:
# test accuracy 77.54
@time accuracy(model,dtst,predict)

  0.551547 seconds (131.39 k allocations: 124.437 MiB, 4.39% gc time)


0.7834535256410257